In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms,utils
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [3]:
data=pd.read_csv('/content/data2.csv')
data=data[['text','sentiment','Measure of Evidence Strength of sent message']]

In [4]:
texts=''.join(data['text'])
texts

"['lucki', 'awesom']['glad', 'make', 'sum', 'one', 'smile']['atom', 'caviti', 'experienc', 'male', 'pattern', 'bold']['jacksonvil', 'ch', 'walk', 'cold', 'w', 'h', 'work', 'morn', 'ly']['pa']['cook', 'meatbal', 'lunch', 'yea']['johnson', 'good', 'night']['find', 'today', 'first', 'kiss']['full', 'time', 'night', 'game', 'ask', 'ton', '0', '5', 'whitney', 'bay']['nice', 'pp', 'realli', 'look']['handi', 'need', 'plan', 'everyth', 'need', 'done', 'blush', '1']['got', 'lock', 'planner', 'enter', 'pa', 'code', 'pp', 'said', 'pass', 'code', 'wrong', 'wrong', 'suspect', 'system', 'pitch', 'contract', 'develop', 'june', '17', 'th', 'year', 'receiv', 'respons', 'yet', 'found', 'way', 'around', 'problem', 'time', 'notr', 'new', 'subscript', 'anymor', 'look', 'anoth', 'pp', 'develop', 'better', 'custom', 'servic']['supernatur', 'good', 'love']['list', 'prioriti', 'calendar', 'view', 'e', 'pp', 'best']['would', 'stop', 'tell', 'give', 'rate', 'henc', 'lower', 'score', 'pretti', 'good', 'pp', 'best

In [5]:
from collections import Counter
vocabulary=texts.split()
word_counts=Counter(vocabulary)
word_counts

Counter({"['lucki',": 1,
         "'awesom']['glad',": 1,
         "'make',": 1079,
         "'sum',": 204,
         "'one',": 1614,
         "'smile']['atom',": 1,
         "'caviti',": 1,
         "'experienc',": 17,
         "'male',": 9,
         "'pattern',": 4,
         "'bold']['jacksonvil',": 1,
         "'ch',": 242,
         "'walk',": 64,
         "'cold',": 108,
         "'w',": 546,
         "'h',": 1092,
         "'work',": 1957,
         "'morn',": 369,
         "'ly']['pa']['cook',": 1,
         "'meatbal',": 1,
         "'lunch',": 100,
         "'yea']['johnson',": 1,
         "'good',": 1614,
         "'night']['find',": 1,
         "'today',": 735,
         "'first',": 350,
         "'kiss']['full',": 1,
         "'time',": 2072,
         "'night',": 598,
         "'game',": 162,
         "'ask',": 652,
         "'ton',": 202,
         "'0',": 60,
         "'5',": 507,
         "'whitney',": 1,
         "'bay']['nice',": 1,
         "'pp',": 4748,
         "'realli'

In [6]:
vocabulary_words= [word for word in vocabulary if word_counts[word]>5]


In [7]:
def avg_word(sentence):
    try:
        words=sentence.split()
        return (sum(len(word) for word in words)/len(words))
    except:
        return 0

In [8]:
avg_word(texts)

8.20252894433212

In [9]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

data['sentiment']=labelencoder.fit_transform(data['sentiment'])
data.head()

text  sentiment  \
0                                ['lucki', 'awesom']          2   
1            ['glad', 'make', 'sum', 'one', 'smile']          2   
2  ['atom', 'caviti', 'experienc', 'male', 'patte...          0   
3  ['jacksonvil', 'ch', 'walk', 'cold', 'w', 'h',...          0   
4                                             ['pa']          2   

   Measure of Evidence Strength of sent message  
0                                      0.901658  
1                                      0.012167  
2                                      0.603517  
3                                      0.814155  
4                                      0.684390

In [10]:
labelencoder.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [11]:
from sklearn.model_selection import train_test_split

train, test=train_test_split(data,random_state=42,test_size=0.2,shuffle=True)
X_train=train.text
X_test=test.text
print(X_train.shape)
print(X_test.shape)

(24985,)
(6247,)


In [12]:
import string

def tokenize (text):
    text=re.sub(r"|^\x00.\x7f]*"," ",text)
    regex=re.compile('('+re.escape(string.punctuation)+'0.9\\r\\t\\n)')
    nopunct=regex.sub("  ",text.lower())
    return text.split()

In [13]:
counts=Counter()
for index, row in data.iterrows():
    counts.update(tokenize(row['text']))

In [14]:
vocab2index={"":0,"UNK":1}
words=["","UNK"]
for word in counts:
    vocab2index[word]=len(words)
    words.append(word)


In [15]:
def encode_sentence(text,vocab2index,N=60):
    tokenized=tokenize(text)
    encoded=np.zeros(N,dtype=int)
    encl=np.array([vocab2index.get(word,vocab2index["UNK"]) for word in tokenized ])
    length=min(N,len(encl))
    encoded[:length]=encl[:length]
    return encoded , length

In [16]:
data['encoded']=data['text'].apply(lambda x:np.array(encode_sentence(x,vocab2index)))
data.head()

<ipython-input-16-ed1e995e6b3c>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data['encoded']=data['text'].apply(lambda x:np.array(encode_sentence(x,vocab2index)))


text  sentiment  \
0                                ['lucki', 'awesom']          2   
1            ['glad', 'make', 'sum', 'one', 'smile']          2   
2  ['atom', 'caviti', 'experienc', 'male', 'patte...          0   
3  ['jacksonvil', 'ch', 'walk', 'cold', 'w', 'h',...          0   
4                                             ['pa']          2   

   Measure of Evidence Strength of sent message  \
0                                      0.901658   
1                                      0.012167   
2                                      0.603517   
3                                      0.814155   
4                                      0.684390   

                                             encoded  
0  [[2, 3, 4, 5, 6, 7, 8, 3, 9, 3, 10, 11, 12, 13...  
1  [[2, 3, 17, 4, 10, 18, 3, 9, 3, 15, 10, 7, 12,...  
2  [[2, 3, 10, 20, 14, 15, 3, 9, 3, 6, 10, 21, 8,...  
3  [[2, 3, 26, 10, 6, 7, 13, 14, 19, 21, 8, 4, 3,...  
4  [[2, 3, 23, 10, 3, 16, 0, 0, 0, 0, 0, 0, 0, 0,...

In [17]:
train,test= train_test_split(data,random_state=42,test_size=0.2,shuffle=True)
X_train=train.encoded
X_test=test.encoded
print(X_train.shape)
print(X_test.shape)

(24985,)
(6247,)


In [18]:
class TextDataset(Dataset):
    def __init__(self,X,Y):
        self.X=X
        self.y=Y

    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)),self.y[idx],self.X[idx][1]


In [19]:
train_ds=TextDataset(X_train.values,train['sentiment'].values)
valid_ds=TextDataset(X_test.values,test['sentiment'].values)


In [20]:
train_ds.__getitem__(0)

(tensor([ 2,  3, 14, 27,  3,  9,  3, 25, 10,  6,  7,  3,  9,  3, 23,  4, 12, 10,
         13,  3,  9,  3, 18, 14, 19, 20,  3,  9,  3, 24,  5,  8, 19,  3, 16,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0], dtype=torch.int32),
 2,
 35)

In [26]:
def train_model(model, epochs,lr):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters,lr=lr)
    best_model=model
    best_accuracy=0
    for i in range (epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x,y,l in train_dl :
                x=x.long()
                y=y.long()
                y_pred = model (x,l)
                optimizer.zero_grad()
                loss=F.cross_entropy(y_pred,y)
                loss.backward()
                optimizer.step()
                sum_loss += loss.item()*y.shape[0]
                total += y.shape[0]
        val_loss,val_acc,val_rmse=validation_metrics(model,val_dl)
        if best_accuracy<val_acc :
          best_accuracy=val_acc
          best_model=model
        print("train loss %.3f, val loss %.3f, val accuracy %.3f, val rmse %.3f" % ((sum_loss/total),val_loss,val_acc,val_rmse))
    return best_model,best_accuracy

def validation_metrics(model,valid_dl):
     model.eval()
     correct=0
     total=0
     sum_loss=0.0
     sum_rmse=0.0
     for x,y,l in valid_dl :
          x=x.long()
          y=y.long()
          y_hat=model(x,l)
          loss=F.cross_entropy(y_hat,y)
          pred=torch.max(y_hat,1)[1]
          correct += (pred==y).float().sum()
          total += y.shape[0]
          sum_loss += loss.item()*y.shape[0]
          sum_rmse += np.sqrt(mean_squared_error(pred,y.unsqueeze(-1)))*y.shape[0]
     return sum_loss/total,correct/total,sum_rmse/total

In [22]:
batch_size=128
vocab_size=len(words)
train_dl=DataLoader(train_ds,batch_size=batch_size,shuffle=True)
val_dl=DataLoader(valid_ds,batch_size=batch_size)

In [23]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings=nn.Embedding(vocab_size,embedding_dim,padding_idx=0)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)

        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.dropout=nn.Dropout(0.2)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.dropout=nn.Dropout(0.2)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.dropout=nn.Dropout(0.2)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)


        self.linear = nn.Linear(hidden_dim,3)
        self.dropout=nn.Dropout(0.2)

    def forward(self,x,l):
        x= self.embeddings(x)
        x= self.dropout(x)
        lstm_out,(ht,ct)=self.lstm(x)
        return self.linear(ht[-1])


In [27]:
model_fixed = LSTM_fixed_len(vocab_size, 40 , 256)
best_model,best_accuracy=train_model(model_fixed,epochs=30,lr=0.01)



train loss 1.099, val loss 1.087, val accuracy 0.384, val rmse 1.043
train loss 1.095, val loss 1.090, val accuracy 0.381, val rmse 0.827
train loss 1.093, val loss 1.087, val accuracy 0.381, val rmse 1.063
train loss 1.085, val loss 1.063, val accuracy 0.427, val rmse 1.041
train loss 1.042, val loss 1.001, val accuracy 0.496, val rmse 0.880
train loss 1.010, val loss 0.995, val accuracy 0.500, val rmse 0.907
train loss 0.991, val loss 0.980, val accuracy 0.516, val rmse 0.840
train loss 0.969, val loss 0.953, val accuracy 0.537, val rmse 0.828
train loss 0.943, val loss 0.939, val accuracy 0.554, val rmse 0.799
train loss 0.924, val loss 0.925, val accuracy 0.559, val rmse 0.775
train loss 0.911, val loss 0.909, val accuracy 0.579, val rmse 0.799
train loss 0.896, val loss 0.918, val accuracy 0.567, val rmse 0.828
train loss 0.882, val loss 0.904, val accuracy 0.573, val rmse 0.817
train loss 0.868, val loss 0.901, val accuracy 0.577, val rmse 0.801
train loss 0.856, val loss 0.899, 

In [ ]:
install

In [7]:

best_model.save("sentiment_analysis.hdl5")

NameError: ignored

NameError: ignored

In [ ]:
model_fixed = LSTM_fixed_len(vocab_size, 40 , 256)
best_model,best_accuracy=train_model(model_fixed,epochs=30,lr=0.01)



train loss 1.099, val loss 1.087, val accuracy 0.384, val rmse 1.043
train loss 1.095, val loss 1.090, val accuracy 0.381, val rmse 0.827
train loss 1.093, val loss 1.087, val accuracy 0.381, val rmse 1.063
train loss 1.085, val loss 1.063, val accuracy 0.427, val rmse 1.041
train loss 1.042, val loss 1.001, val accuracy 0.496, val rmse 0.880
train loss 1.010, val loss 0.995, val accuracy 0.500, val rmse 0.907
train loss 0.991, val loss 0.980, val accuracy 0.516, val rmse 0.840
train loss 0.969, val loss 0.953, val accuracy 0.537, val rmse 0.828
train loss 0.943, val loss 0.939, val accuracy 0.554, val rmse 0.799
train loss 0.924, val loss 0.925, val accuracy 0.559, val rmse 0.775
train loss 0.911, val loss 0.909, val accuracy 0.579, val rmse 0.799
train loss 0.896, val loss 0.918, val accuracy 0.567, val rmse 0.828
train loss 0.882, val loss 0.904, val accuracy 0.573, val rmse 0.817
train loss 0.868, val loss 0.901, val accuracy 0.577, val rmse 0.801
train loss 0.856, val loss 0.899, 

In [8]:
best_model.save('model.hdf5')

NameError: ignored

In [ ]:
train_df=pd.DataFrame({'text':train['text'],'label':train['sentiment'].astype(int)})

train_df.head()

text  label
19163            ['oh', 'back', 'pleas', 'dont', 'ruin']      2
7071             ['cm', 'peopl', 'today', 'judi', 'day']      1
6872   ['get', 'new', 'password', 'send', 'messag', '...      0
28745   ['add', 'list', 'featur', 'highli', 'recommend']      1
29242                             ['public', 'bathroom']      1

In [ ]:
from simpletransformers.classification import ClassificationModel



model = ClassificationModel('roberta','roberta-base',num_labels=3,args={'learning_rate':1e-5,'num_train_epochs': 10,'reprocess_input_data':True,'overwrite_output_dir':True}, use_cuda=False)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
model.train_model(train_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/24985 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/3124 [00:00<?, ?it/s]

KeyboardInterrupt: ignored